In [3]:
import numpy as np
import os
import csv
import math

In [11]:
#For personal use!
#Functions that use different scoring methods in order to be able to compare our own CT scoring system against other conventional methods
#contains all proteinnames and their respective ID's

protidlist = []

with open('results/statistics/psc/pscresults'+count+'.txt') as f:
    for line in f:
        protidlist.append(line.split(None,1)[0])

protidarr = np.array(protidlist,dtype='str')

#The number of parallel(p), series(s) and cross(c) contacts in a protein 
psclist = np.genfromtxt('results/statistics/psc/pscresults'+count+'.txt',delimiter=' ',dtype='double')[:,1:]

row_sum = np.sum(psclist,axis = 1)
idx = np.flatnonzero(row_sum)

psclist = psclist[idx]
protidarr = protidarr[idx]

#Calculates the avg. amount of P, S and C.
mean = np.mean(psclist,axis=0)
mean = np.true_divide(mean,np.sum(mean))

#Utilizes Jaie's square method to calculate a score based on the sum of the squared difference in fractions of relations
for i,row in enumerate(psclist):
    row = np.true_divide(row,np.sum(row))
    psclist[i] = row


#compared to the average
jscore_avg =  np.round(1-np.sqrt((np.sum((mean-psclist)**2,axis=1))),3)

#pairwise calculation
jscore_pairwise = np.ones([len(psclist),len(psclist)])

def topology_dis(protid1,protid2):
    prot1 = psclist[protid1]
    prot2 = psclist[protid2]
    score = np.round(1-np.sqrt((np.sum((prot1-prot2)**2)))/np.sqrt(2),3)
    return score

for i in range(0,len(psclist)):
    for j in range(0,len(psclist)):
        jscore_pairwise[i][j] = topology_dis(i,j)

np.fill_diagonal(jscore_pairwise, np.nan)



In [107]:
#import tm-align scores from external program
tmalign = np.genfromtxt('/Users/duanemoes/Desktop/Desktop/RP1/Code/Tm_align/tmalign.txt',dtype='str',delimiter=',')
tmalign = np.char.strip(tmalign)
tmalignname = np.array(tmalign[:,0:2],dtype='str')
tmalignscore = np.array(tmalign[:,2:],dtype='double')

#Transposes the data into an array
tmalignarr= np.zeros([len(protidarr),len(protidarr)],dtype='double')

for i,row in enumerate(tmalignname):
    prot2 = np.where(protidarr==row[1])[0]
    prot1 = np.where(protidarr==row[0])[0]
    prot1score = tmalignscore[i,0]
    prot2score = tmalignscore[i,1]
    tmalignarr[prot1,prot2] = prot1score
    tmalignarr[prot2,prot1] = prot2score

#Calculates the difference between the TM-align score and the CT score
def sorta(ls):
    ls.sort(key = lambda x: x[2])
    return ls
    
difference = []

for i in range(0,len(jscore_pairwise)):
    for j in range(0,len(jscore_pairwise)):
        if i == j:
            continue
        rslt = (jscore_pairwise[i][j] - tmalignarr[i][j])**2
        difference.append([i,j,rslt])

difference = np.array(sorta(difference),dtype=None)

#Finds the 10 proteins that are closest to eachother and 10 that differ the most
top10difference = difference[np.r_[0:10,-10:0]]

#Finds matching protein names
top10protls = []

for row in top10difference:
    prot1 = protidarr[int(row[0])]
    prot2 = protidarr[int(row[1])]
    top10protls.append([prot1, prot2])

#combined normalized scores
scores = np.zeros([20,5],dtype=object)
for i,row in enumerate(top10difference):
    scores[i,0] = top10protls[i][0]
    scores[i,1] = top10protls[i][1]
    scores[i,2] = jscore_pairwise[int(row[0]),int(row[1])]
    scores[i,3] = tmalignarr[int(row[0]),int(row[1])]
    scores[i,4] = pdbefoldnew[i][2]

np.savetxt("combinedscores.csv",scores,delimiter=';',fmt='%s,%s,%.5f,%.5f,%.5f')

In [108]:
#Calculates the difference between the TM-align score and the CT score
def sorta(ls):
    ls.sort(key = lambda x: x[2])
    return ls
    
difference = []

for i in range(0,len(jscore_pairwise)):
    for j in range(0,len(jscore_pairwise)):
        if i == j:
            continue
        rslt = (jscore_pairwise[i][j] - tmalignarr[i][j])**2
        difference.append([i,j,rslt])

difference = np.array(sorta(difference),dtype=None)

#Finds the 10 proteins that are closest to eachother and 10 that differ the most
top10difference = difference[np.r_[0:10,-10:0]]

#Finds matching protein names
top10protls = []

for row in top10difference:
    prot1 = protidarr[int(row[0])]
    prot2 = protidarr[int(row[1])]
    top10protls.append([prot1, prot2])

#combined normalized scores
scores = np.zeros([20,5],dtype=object)
for i,row in enumerate(top10difference):
    scores[i,0] = top10protls[i][0]
    scores[i,1] = top10protls[i][1]
    scores[i,2] = jscore_pairwise[int(row[0]),int(row[1])]
    scores[i,3] = tmalignarr[int(row[0]),int(row[1])]
    scores[i,4] = pdbefoldnew[i][2]

np.savetxt("combinedscores.csv",scores,delimiter=';',fmt='%s,%s,%.5f,%.5f,%.5f')


In [111]:
#PDBefold scores
#Creates a txt files with all the top 10 pairs, to be used as input in PDBefold
with open('/Users/duanemoes/Desktop/Desktop/RP1/Code/pdbefold/ls.txt',"w") as f:
    for row in top10protls:
        f.write(f'{row[0]}.pdb {row[1]}.pdb\n')

#PDBefold output scores 
pdbefold = []
with open('/Users/duanemoes/Desktop/Desktop/RP1/Code/pdbefold/results.txt') as f:
    for line in f:
        x = line.split()
        pdbefold.append([x[1],x[2],x[0]])

for i in range(0,len(pdbefold)):
    pdbefold[i][0] = list(np.where(protidarr == pdbefold[i][0])[0])[0]
    pdbefold[i][1] = list(np.where(protidarr == pdbefold[i][1])[0])[0]
    #here we normalize the scores from range [0,16] --> [0,1]
    pdbefold[i][2] = float(pdbefold[i][2])/16

pdbefold = np.array(pdbefold)

#Sorts pdbefold values based on top 10 difference results
toptest = np.sum(top10difference[:,0:2],axis=1)
pdbtest1 = np.sum(pdbefold[:,0:2],axis=1)

pdbefoldnew = np.array(pdbefold)
for i in range(0,len(pdbefold)):
    x = np.where(toptest == pdbtest1[i])
    pdbefoldnew[x] = pdbefold[i] 

In [112]:
#combined normalized scores
scores = np.zeros([20,5],dtype=object)
for i,row in enumerate(top10difference):
    scores[i,0] = top10protls[i][0]
    scores[i,1] = top10protls[i][1]
    scores[i,2] = jscore_pairwise[int(row[0]),int(row[1])]
    scores[i,3] = tmalignarr[int(row[0]),int(row[1])]
    scores[i,4] = pdbefoldnew[i][2]

np.savetxt("combinedscores.csv",scores,delimiter=';',fmt='%s,%s,%.5f,%.5f,%.5f')

In [35]:
#6 randomized sets of 20 proteins selected for PDBefold
sets = []
sets.append(np.random.choice(protidlist,20))
sets.append(np.random.choice(protidlist,20))
sets.append(np.random.choice(protidlist,20))
sets.append(np.random.choice(protidlist,20))
sets.append(np.random.choice(protidlist,20))
sets.append(np.random.choice(protidlist,20))


In [45]:
#Create 5 separte pair files to be used for PDBefold
for a in range(0,len(sets)):
    f = open("set%i.txt" % a, "w")
    for i in range(0,len(set1)):
        for j in range(0,len(set1)):
            if i == j:
                continue
            f.write("%s.pdb %s.pdb\n" % (sets[a][i],sets[a][j]))    

0
1
2
3
4
5


In [56]:
pdbefoldrandom = []
for x in range(0,6):
    with open('/Users/duanemoes/Desktop/Desktop/RP1/Code/pdbefold/results%i.txt' % x) as f:
        for line in f:
            a = line.split()
            a[-2] = a[-2][0:4]
            a[-1] = a[-1][0:4]
            pdbefoldrandom.append([a[-2],a[-1],a[3]])
            
protpdb = np.array(pdbefoldrandom)[:,:2]

for i in range(0,6):
    with open('/Users/duanemoes/Desktop/Desktop/RP1/Code/pdbefold/set%i.txt' % i) as f:
        for line in f:
            a = line.split()
            a[0] = a[0][0:4]
            a[1] = a[1][0:4]
            if a in protpdb:
                continue
            else:
                print('no')
            #if a not in protpdb:
                #print('found')
                #pdbefoldrandom.append([a[0],a[1],0])
        
